# Advanced Acquisition

This section explains some of the more advanced things you can do with the `acquire` module.


## pre_hook=

A pre hook is an optional keyword of type callable that you can provide to any `acquire` methods that:

- does something with the argument you pass as source
- passes back a variable that will be used as the new source source by "acquire".

The code is [here](https://github.com/mikeAdamss/datachef/blob/ece2764eba4d7b19fa3817de75c0fc0e71b6de68/datachef/acquire/main.py#L38) and is not complicated (it's literally two lines).

It is however a _very useful_ two lines, examples follow:

### Example Scenario

For our example scenario we're going to use a **url to a json api** (a tiny one we've made ourselves) as a source.

You can view the contents of the api [here](https://raw.githubusercontent.com/mikeAdamss/datachef/main/tests/fixtures/json/api.json).

Our requirements are as follows:

- We need to get the url of our source from that api.
- The api for the json api will **not** change.
- The urls listed **will** change.
- We want to always acquire the url with "bands-wide" in the path, there will always be one but the numher of other urls and its positioning within that list will change.
- We want an exception if there's anything other than exactly one "bands-wide" url.
- We want that "bands-wide" url to be passed along to the `acquire` function and used to create our selectable.

Example follows:

In [3]:
import requests

# First we'll create a simple pre hook that meets our requirements.
def select_band_csv(source: str) -> str:
    """
    pre hook function to get the correct url from a list of urls
    provided by a json api.
    """
    r = requests.get(source)
    url_dict= r.json()
    url_wanted = [x for x in url_dict["datasets"] if "bands-wide" in x]
    assert len(url_wanted) == 1
    return url_wanted[0]

# Now let's try it out
select_band_csv("https://raw.githubusercontent.com/mikeAdamss/datachef/main/tests/fixtures/json/api.json")

'https://raw.githubusercontent.com/mikeAdamss/datachef/main/tests/fixtures/csv/bands-wide.csv'

And now let's see how we use this re hook as a keyword argument to the acquire module.

In [4]:
from datachef import acquire, preview, CsvSelectable

table: CsvSelectable = acquire.csv.http(
    "https://raw.githubusercontent.com/mikeAdamss/datachef/main/tests/fixtures/json/api.json",
    pre_hook=select_band_csv)

preview(table)


,A,B,C,D,E,F,G,H,I,J,K
1,,,,,,,,,,,
2,,,Houses,Cars,Boats,,,,Houses,Cars,Boats
3,Beatles,,,,,,Rolling Stones,,,,
4,,John,1,5,9,,,Keith,2,6,10
5,,Paul,2,6,10,,,Mick,3,7,11
6,,George,2,7,11,,,Charlie,3,8,12
7,,Ringo,4,8,12,,,Ronnie,5,9,13
8,,,,,,,,,,,


And voila! We're dynamically selecting the csv we want from the options presented by a json api.

| <span style="color:green">Note - as per datachef standard approch we wouldn't necessarily recommend you write these in line (though you can) but rather we'd recommend that pre-hooks of this source should be created and reused across your ETL processes.</span>|
|-----------------------------------------|